In [1]:
import numpy as np
import glob
import os
from opac_mixer.read import ReadOpacChubb
from opac_mixer.emulator import Emulator

import matplotlib.pyplot as plt

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error
from ipywidgets import interact, IntSlider, FloatSlider, Dropdown
%matplotlib widget

In [2]:
R='S1'
base = f'{os.environ["pRT_input_data_path"]}/opacities/lines/corr_k'
files = glob.glob(os.path.join(base,f'*_R_{R}/*.h5'))
batchsize = int(2**16)

load = True
load_model = False

In [3]:
opac = ReadOpacChubb(files)
emulator = Emulator(opac)

In [4]:
emulator.setup_sampling_grid(batchsize=batchsize, filename=f'data/input_{batchsize}', load=load)
X_train, X_test, y_train, y_test = emulator.setup_mix(filename=f'data/mix_R_{R}_{batchsize}', load=load)

In [5]:
emulator.setup_model(load=load_model, filename=f'data/model_R_{R}_{batchsize}', tree_method='hist', n_estimators=100)

In [6]:
emulator.fit()

[0]	validation_0-rmse:5.35395	validation_1-rmse:5.34885
[1]	validation_0-rmse:3.81519	validation_1-rmse:3.81163
[2]	validation_0-rmse:2.74136	validation_1-rmse:2.73899
[3]	validation_0-rmse:1.99487	validation_1-rmse:1.99457
[4]	validation_0-rmse:1.47886	validation_1-rmse:1.48149
[5]	validation_0-rmse:1.12414	validation_1-rmse:1.13051
[6]	validation_0-rmse:0.88206	validation_1-rmse:0.89382
[7]	validation_0-rmse:0.71756	validation_1-rmse:0.73617
[8]	validation_0-rmse:0.60691	validation_1-rmse:0.63274
[9]	validation_0-rmse:0.53038	validation_1-rmse:0.56382
[10]	validation_0-rmse:0.47622	validation_1-rmse:0.51775


In [ ]:
y_predict_test = emulator.predict(emulator.X_test, shape='same')
y_predict_test_scaled = emulator.model.predict(emulator.X_test)
print('r2score on test:', r2_score(y_true=emulator.y_test, y_pred=y_predict_test))
print('mean squared error on test:', np.sqrt(mean_squared_error(y_true=emulator.y_test, y_pred=y_predict_test)))
print('mean absolute percentage error on :', mean_absolute_percentage_error(y_true=emulator.y_test, y_pred=y_predict_test))

print('r2score on scaled test:', r2_score(y_true=emulator.output_scaling(emulator.y_test), y_pred=y_predict_test_scaled))
print('mean squared error on scaled test:', np.sqrt(mean_squared_error(y_true=emulator.output_scaling(emulator.y_test), y_pred=y_predict_test_scaled)))
print('mean absolute percentage error on scaled test:', mean_absolute_percentage_error(y_true=emulator.output_scaling(emulator.y_test), y_pred=y_predict_test_scaled))

AttributeError: we do not have a trained model yet. Run setup_sampling_grid, setup_mix and setup_model and fit first.

In [ ]:
y_predict_train = emulator.predict(emulator.X_train, shape='same')
y_predict_train_scaled = emulator.model.predict(emulator.X_train)

print('r2score on train:', r2_score(y_true=emulator.y_train, y_pred=y_predict_train))
print('mean squared error on train:', np.sqrt(mean_squared_error(y_true=emulator.y_train, y_pred=y_predict_train)))
print('mean absolute percentage error on train:', mean_absolute_percentage_error(y_true=emulator.y_train, y_pred=y_predict_train))

print('r2score on scaled train:', r2_score(y_true=emulator.output_scaling(emulator.y_train), y_pred=y_predict_train_scaled, force_finite=True))
print('mean absolute error on scaled train:', np.sqrt(mean_squared_error(y_true=emulator.output_scaling(emulator.y_train), y_pred=y_predict_train_scaled)))
print('mean absolute percentage error on scaled train:', mean_absolute_percentage_error(y_true=emulator.output_scaling(emulator.y_train), y_pred=y_predict_train_scaled))

r2score on train: 0.9837302595920356
mean squared error on train: 1.9463812201200943
mean absolute percentage error on train: 0.41704834612375974
r2score on scaled train: 0.994299296994706
mean absolute error on scaled train: 0.20912797675454944
mean absolute percentage error on scaled train: 0.05420733117638952


In [ ]:
y_predict = {'test': emulator.reshape(y_predict_test), 'train': emulator.reshape(y_predict_train)}
y_true = {'test':emulator.reshape(emulator.y_test), 'train': emulator.reshape(emulator.y_train)}

In [ ]:
fig_tes, ax_tes = plt.subplots(1,1)
def plot_single_prediction(index_frac, mode):
    ax_tes.cla()
    index = np.array([int(index_frac*len(y_predict[mode]))])
    for fi in range(y_predict[mode].shape[1]):
        x = opac.bin_edges[fi]+opac.weights.cumsum()*(opac.bin_edges[fi+1]-opac.bin_edges[fi])
        lp, = ax_tes.loglog(x,y_predict[mode][index][0,fi,:], color ='red', alpha=0.5)
        lt, = ax_tes.loglog(x,y_true[mode][index][0,fi,:], color = 'blue', alpha=0.5)
        
    # ax_tes.set_yscale('linear')
    ax_tes.set_ylabel(r'$\kappa$ [cm2/g]')
    ax_tes.set_xlabel(r'wavenumber')
    ax_tes.set_title(f'{mode} data, {index}')
    ax_tes.legend([lp,lt],['prediction','true'])

interact(plot_single_prediction, index_frac=FloatSlider(min=0., max=.99, step=0.05, value=0.), mode=Dropdown(options=['train','test']));

interactive(children=(FloatSlider(value=0.0, description='index_frac', max=0.99, step=0.05), Dropdown(descript…

In [ ]:
fig_t, ax_t = plt.subplots(1,1) 
lt, = ax_t.plot(y_predict['test'][:,0,0], y_true['test'][:,0,0], 'ro', ms=0.01, linestyle="None")
ax_t.set_xscale('log')
ax_t.set_yscale('log')
dt, = ax_t.plot([y_predict['test'][:,0,0].min(), y_predict['test'][:,0,0].max()], [y_predict['test'][:,0,0].min(), y_predict['test'][:,0,0].max()], color = 'gray', ls = '--')
ax_t.set_ylabel('true values')
ax_t.set_xlabel('predicted values')
    

def update_prediction_test(freqi, gi, mode):
    lt.set_ydata(y_true[mode][:,freqi,gi])
    lt.set_xdata(y_predict[mode][:,freqi,gi])    
    dt.set_ydata([y_predict[mode][:,freqi,gi].min(), y_predict[mode][:,freqi,gi].max()])
    dt.set_xdata([y_predict[mode][:,freqi,gi].min(), y_predict[mode][:,freqi,gi].max()])
    ax_t.set_xlim([y_true[mode][:,freqi,gi].min(),y_true[mode][:,freqi,gi].max()])
    ax_t.set_ylim([y_true[mode][:,freqi,gi].min(),y_true[mode][:,freqi,gi].max()])        
    ax_t.set_title(f'{mode} data variance')
    fig_t.canvas.draw_idle()


interact(update_prediction_test, freqi=IntSlider(min=0, max=opac.lf[0]-1, step=1, value=0),gi=IntSlider(min=0, max=opac.lg[0]-1, step=1, value=0), mode=Dropdown(options=['train','test']));

interactive(children=(IntSlider(value=0, description='freqi', max=9), IntSlider(value=0, description='gi', max…